In [1]:
# -*- coding: utf-8 -*-
import time
import os
import tensorflow as tf
import numpy as np
%run Bi_LSTM.py
%run Word2Vec.py
import pandas as pd

C:\Users\dongc\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 여기부터 하면 됨

In [2]:
from tqdm import tqdm_notebook
from eunjeon import Mecab
from konlpy.tag import Okt

In [3]:
mecab = Mecab()
okt = Okt()
W2V = Word2Vec()

In [4]:
df = pd.read_csv('../data/자소서_잡코추가2.csv',index_col=0, engine='python', encoding='utf8')

## 데이터 섞기

In [5]:
df = df.reset_index().drop(columns={'index'})

In [6]:
rows = np.random.choice(df.index.values, len(df), replace=False)
sampled_df = df.loc[rows]

In [7]:
df = sampled_df

In [8]:
dataset = int(len(df) * 0.7)

In [9]:
df.iloc[:dataset].groupby('category').agg('count')

,c_origin,jss,label
category,,,
경영지원,3761,3761,3761
생산·물류,3135,3135,3135
서버·시스템,1215,1215,1215
영업·마케팅,6814,6814,6814
응용프로그래밍,2207,2207,2207
인사,534,534,534
재무·금융,2317,2317,2317
통계,156,156,156


In [10]:
df.head()

,category,c_origin,jss,label
26132,영업·마케팅,마케팅·광고·분석,[Life is Sweet]\n\n에뛰드 매장 특유의 달콤한 향기 그리고 특징적인 ...,3
23403,영업·마케팅,영업관리·지원·영업기획,HanSae Fashion Worldwide\nHuman : 사람을 좋아하는 저의 ...,3
11530,경영지원,경영기획,"- 경험의 배경, 본인의 역할, 동료와 도움을 주고받은 부분, 당시 감정이나 생각,...",0
28467,응용프로그래밍,IT·디자인·컴퓨터강사,소통하면서 개발을 하기 위해 삼성전자에 지원했습니다. SW개발은 창의성 발현에서 동...,7
16097,경영지원,기획·전략·경영,"악기를 배움으로써 길렀던 끈기, 대표를 맡으면서 얻게 된 쉽게 포기하지 않으려는 책...",0


In [11]:
data = []
for txt in tqdm_notebook(df.jss):
    data.append(okt.nouns(txt))

In [12]:
labels = df.label

# 워드 임베딩

In [13]:
config = tf.ConfigProto()
tf.reset_default_graph()
W2V = Word2Vec()

In [14]:
dataset = int(len(data)*0.7)

In [15]:
# 70% train, 30% test
train_X = data[:dataset]
train_Y = labels[:dataset]
test_X = data[dataset:]
test_Y = labels[dataset:]

In [16]:
nb_classes = len(df.category.unique())

In [17]:
train_X_ = W2V.Convert2Vec("../data/word2vec_okt_nouns2.model",train_X)
test_X_ = W2V.Convert2Vec("../data/word2vec_okt_nouns2.model",test_X)

In [18]:
train_Y_ = W2V.One_hot(train_Y)
test_Y_ = W2V.One_hot(test_Y)

# 파라미터

In [19]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300 # 모델의 워드벡터 사이즈
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]
Maxseq_length = max(max(train_seq_length), max(test_seq_length))
learning_rate = 0.001
lstm_units = 128
num_class = nb_classes
training_epochs = 9

# 그래프 만들기

In [20]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])
keep_prob = tf.placeholder(tf.float32, shape = None)

In [21]:
BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

In [22]:
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
total_batch = int(len(train_X) / Batch_size)
test_batch = int(len(test_X) / Batch_size)

In [24]:
modelName = "./Bidirectional_LSTM_4/BiLSTM_model.ckpt"
saver = tf.train.Saver()

train_acc = []
train_loss = []
test_acc = []
test_loss = []

# 세션 런

In [25]:
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
start_time = time.time()
sess.run(init)
train_writer = tf.summary.FileWriter('./Bidirectional_LSTM_4', sess.graph)
merged = BiLSTM.graph_build()

In [26]:
for epoch in range(training_epochs):  

    avg_acc, avg_loss = 0. , 0.
    mask = np.random.permutation(len(train_X_))
    train_X_ = train_X_[mask]
    train_Y_ = train_Y_[mask]

    for step in tqdm_notebook(range(total_batch)):

        train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
        train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = train_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)

        sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
        # Compute average loss
        loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 0.75})
        avg_loss += loss_ / total_batch

        acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                             keep_prob : 0.75})
        avg_acc += acc / total_batch
        print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

    summary = sess.run(merged, feed_dict = {BiLSTM.loss : avg_loss, BiLSTM.acc : avg_acc})       
    train_writer.add_summary(summary, epoch)

    t_avg_acc, t_avg_loss = 0. , 0.
    print(avg_loss, avg_acc)
    print("Test cases, could take few minutes")
    
    for step in tqdm_notebook(range(test_batch)):

        test_batch_X = test_X_[step*Batch_size : step*Batch_size+Batch_size]
        test_batch_Y = test_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = test_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        test_batch_X = W2V.Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)

        # Compute average loss
        loss2 = sess.run(loss, feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 1.0})
        t_avg_loss += loss2 / test_batch

        t_acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                               keep_prob : 1.0})
        t_avg_acc += t_acc / test_batch

    print("<Train> Loss = {:.6f} Accuracy = {:.6f}".format(avg_loss, avg_acc))
    print("<Test> Loss = {:.6f} Accuracy = {:.6f}".format(t_avg_loss, t_avg_acc))
    train_loss.append(avg_loss)
    train_acc.append(avg_acc)
    test_loss.append(t_avg_loss)
    test_acc.append(t_avg_acc)

epoch : 01 step : 0001 loss = 1.542298 accuracy= 0.718750
epoch : 01 step : 0002 loss = 1.605308 accuracy= 0.562500
epoch : 01 step : 0003 loss = 1.733656 accuracy= 0.406250
epoch : 01 step : 0004 loss = 1.744917 accuracy= 0.343750
epoch : 01 step : 0005 loss = 1.574064 accuracy= 0.406250
epoch : 01 step : 0006 loss = 1.718332 accuracy= 0.343750
epoch : 01 step : 0007 loss = 1.784848 accuracy= 0.343750
epoch : 01 step : 0008 loss = 1.941540 accuracy= 0.187500
epoch : 01 step : 0009 loss = 1.516667 accuracy= 0.468750
epoch : 01 step : 0010 loss = 1.507486 accuracy= 0.562500
epoch : 01 step : 0011 loss = 1.872974 accuracy= 0.312500
epoch : 01 step : 0012 loss = 1.882597 accuracy= 0.281250
epoch : 01 step : 0013 loss = 1.698654 accuracy= 0.406250
epoch : 01 step : 0014 loss = 1.914487 accuracy= 0.218750
epoch : 01 step : 0015 loss = 1.710931 accuracy= 0.312500
epoch : 01 step : 0016 loss = 1.724512 accuracy= 0.375000
epoch : 01 step : 0017 loss = 1.657272 accuracy= 0.437500
epoch : 01 ste

epoch : 01 step : 0142 loss = 1.463920 accuracy= 0.500000
epoch : 01 step : 0143 loss = 1.237006 accuracy= 0.531250
epoch : 01 step : 0144 loss = 1.560930 accuracy= 0.406250
epoch : 01 step : 0145 loss = 1.589747 accuracy= 0.406250
epoch : 01 step : 0146 loss = 1.215442 accuracy= 0.437500
epoch : 01 step : 0147 loss = 1.197388 accuracy= 0.625000
epoch : 01 step : 0148 loss = 1.387430 accuracy= 0.437500
epoch : 01 step : 0149 loss = 1.566783 accuracy= 0.406250
epoch : 01 step : 0150 loss = 1.374342 accuracy= 0.500000
epoch : 01 step : 0151 loss = 1.529785 accuracy= 0.437500
epoch : 01 step : 0152 loss = 1.611046 accuracy= 0.500000
epoch : 01 step : 0153 loss = 1.411108 accuracy= 0.437500
epoch : 01 step : 0154 loss = 1.300421 accuracy= 0.562500
epoch : 01 step : 0155 loss = 1.505171 accuracy= 0.437500
epoch : 01 step : 0156 loss = 1.670755 accuracy= 0.375000
epoch : 01 step : 0157 loss = 1.388287 accuracy= 0.468750
epoch : 01 step : 0158 loss = 1.152080 accuracy= 0.562500
epoch : 01 ste

epoch : 01 step : 0283 loss = 1.299662 accuracy= 0.562500
epoch : 01 step : 0284 loss = 1.228865 accuracy= 0.562500
epoch : 01 step : 0285 loss = 1.478662 accuracy= 0.437500
epoch : 01 step : 0286 loss = 1.454156 accuracy= 0.500000
epoch : 01 step : 0287 loss = 1.110780 accuracy= 0.468750
epoch : 01 step : 0288 loss = 1.071436 accuracy= 0.593750
epoch : 01 step : 0289 loss = 1.372881 accuracy= 0.437500
epoch : 01 step : 0290 loss = 1.353163 accuracy= 0.531250
epoch : 01 step : 0291 loss = 1.400580 accuracy= 0.500000
epoch : 01 step : 0292 loss = 1.249742 accuracy= 0.531250
epoch : 01 step : 0293 loss = 1.159539 accuracy= 0.718750
epoch : 01 step : 0294 loss = 1.650498 accuracy= 0.437500
epoch : 01 step : 0295 loss = 1.559486 accuracy= 0.437500
epoch : 01 step : 0296 loss = 1.192463 accuracy= 0.593750
epoch : 01 step : 0297 loss = 1.435017 accuracy= 0.468750
epoch : 01 step : 0298 loss = 1.384417 accuracy= 0.531250
epoch : 01 step : 0299 loss = 1.246391 accuracy= 0.562500
epoch : 01 ste

epoch : 01 step : 0424 loss = 1.447928 accuracy= 0.531250
epoch : 01 step : 0425 loss = 1.353660 accuracy= 0.531250
epoch : 01 step : 0426 loss = 1.210871 accuracy= 0.593750
epoch : 01 step : 0427 loss = 1.343975 accuracy= 0.531250
epoch : 01 step : 0428 loss = 1.292602 accuracy= 0.531250
epoch : 01 step : 0429 loss = 1.113557 accuracy= 0.562500
epoch : 01 step : 0430 loss = 1.116963 accuracy= 0.531250
epoch : 01 step : 0431 loss = 1.125508 accuracy= 0.625000
epoch : 01 step : 0432 loss = 1.041148 accuracy= 0.531250
epoch : 01 step : 0433 loss = 1.330382 accuracy= 0.468750
epoch : 01 step : 0434 loss = 1.417994 accuracy= 0.500000
epoch : 01 step : 0435 loss = 1.471501 accuracy= 0.562500
epoch : 01 step : 0436 loss = 1.308448 accuracy= 0.468750
epoch : 01 step : 0437 loss = 1.095048 accuracy= 0.687500
epoch : 01 step : 0438 loss = 1.246887 accuracy= 0.562500
epoch : 01 step : 0439 loss = 1.451471 accuracy= 0.437500
epoch : 01 step : 0440 loss = 1.265589 accuracy= 0.562500
epoch : 01 ste

epoch : 01 step : 0565 loss = 1.053293 accuracy= 0.593750
epoch : 01 step : 0566 loss = 1.415683 accuracy= 0.531250
epoch : 01 step : 0567 loss = 1.141662 accuracy= 0.562500
epoch : 01 step : 0568 loss = 1.451440 accuracy= 0.437500
epoch : 01 step : 0569 loss = 1.260192 accuracy= 0.468750
epoch : 01 step : 0570 loss = 1.146137 accuracy= 0.625000
epoch : 01 step : 0571 loss = 1.136192 accuracy= 0.593750
epoch : 01 step : 0572 loss = 1.188719 accuracy= 0.531250
epoch : 01 step : 0573 loss = 1.753735 accuracy= 0.437500
epoch : 01 step : 0574 loss = 1.101742 accuracy= 0.593750
epoch : 01 step : 0575 loss = 1.074693 accuracy= 0.625000
epoch : 01 step : 0576 loss = 1.520284 accuracy= 0.375000
epoch : 01 step : 0577 loss = 1.106121 accuracy= 0.531250
epoch : 01 step : 0578 loss = 1.318750 accuracy= 0.562500
epoch : 01 step : 0579 loss = 1.395746 accuracy= 0.562500
epoch : 01 step : 0580 loss = 1.103003 accuracy= 0.468750
epoch : 01 step : 0581 loss = 1.542019 accuracy= 0.468750
epoch : 01 ste

...

epoch : 09 step : 0142 loss = 0.247223 accuracy= 0.906250
epoch : 09 step : 0143 loss = 0.168347 accuracy= 0.937500
epoch : 09 step : 0144 loss = 0.300638 accuracy= 0.937500
epoch : 09 step : 0145 loss = 0.129295 accuracy= 0.968750
epoch : 09 step : 0146 loss = 0.061631 accuracy= 1.000000
epoch : 09 step : 0147 loss = 0.150170 accuracy= 0.968750
epoch : 09 step : 0148 loss = 0.332181 accuracy= 0.906250
epoch : 09 step : 0149 loss = 0.110419 accuracy= 1.000000
epoch : 09 step : 0150 loss = 0.177824 accuracy= 0.968750
epoch : 09 step : 0151 loss = 0.200930 accuracy= 0.937500
epoch : 09 step : 0152 loss = 0.159590 accuracy= 0.968750
epoch : 09 step : 0153 loss = 0.303036 accuracy= 0.937500
epoch : 09 step : 0154 loss = 0.107156 accuracy= 0.968750
epoch : 09 step : 0155 loss = 0.131591 accuracy= 1.000000
epoch : 09 step : 0156 loss = 0.089002 accuracy= 1.000000
epoch : 09 step : 0157 loss = 0.270121 accuracy= 0.906250
epoch : 09 step : 0158 loss = 0.144640 accuracy= 1.000000
epoch : 09 ste

epoch : 09 step : 0283 loss = 0.124790 accuracy= 0.968750
epoch : 09 step : 0284 loss = 0.233760 accuracy= 0.937500
epoch : 09 step : 0285 loss = 0.154281 accuracy= 0.968750
epoch : 09 step : 0286 loss = 0.302647 accuracy= 0.937500
epoch : 09 step : 0287 loss = 0.136127 accuracy= 0.968750
epoch : 09 step : 0288 loss = 0.145620 accuracy= 1.000000
epoch : 09 step : 0289 loss = 0.071243 accuracy= 1.000000
epoch : 09 step : 0290 loss = 0.284591 accuracy= 0.906250
epoch : 09 step : 0291 loss = 0.117992 accuracy= 1.000000
epoch : 09 step : 0292 loss = 0.198748 accuracy= 0.937500
epoch : 09 step : 0293 loss = 0.208316 accuracy= 0.906250
epoch : 09 step : 0294 loss = 0.228590 accuracy= 0.937500
epoch : 09 step : 0295 loss = 0.164352 accuracy= 0.968750
epoch : 09 step : 0296 loss = 0.101786 accuracy= 1.000000
epoch : 09 step : 0297 loss = 0.419313 accuracy= 0.843750
epoch : 09 step : 0298 loss = 0.177143 accuracy= 0.968750
epoch : 09 step : 0299 loss = 0.158319 accuracy= 0.937500
epoch : 09 ste

epoch : 09 step : 0424 loss = 0.193745 accuracy= 0.937500
epoch : 09 step : 0425 loss = 0.244767 accuracy= 0.937500
epoch : 09 step : 0426 loss = 0.136334 accuracy= 0.968750
epoch : 09 step : 0427 loss = 0.223953 accuracy= 0.937500
epoch : 09 step : 0428 loss = 0.155423 accuracy= 0.968750
epoch : 09 step : 0429 loss = 0.261690 accuracy= 0.937500
epoch : 09 step : 0430 loss = 0.195092 accuracy= 0.968750
epoch : 09 step : 0431 loss = 0.158263 accuracy= 0.968750
epoch : 09 step : 0432 loss = 0.200884 accuracy= 0.906250
epoch : 09 step : 0433 loss = 0.155623 accuracy= 0.937500
epoch : 09 step : 0434 loss = 0.184605 accuracy= 0.906250
epoch : 09 step : 0435 loss = 0.161822 accuracy= 1.000000
epoch : 09 step : 0436 loss = 0.170414 accuracy= 0.937500
epoch : 09 step : 0437 loss = 0.191427 accuracy= 0.968750
epoch : 09 step : 0438 loss = 0.212333 accuracy= 0.937500
epoch : 09 step : 0439 loss = 0.347428 accuracy= 0.875000
epoch : 09 step : 0440 loss = 0.276929 accuracy= 0.906250
epoch : 09 ste

epoch : 09 step : 0565 loss = 0.174230 accuracy= 0.937500
epoch : 09 step : 0566 loss = 0.523476 accuracy= 0.812500
epoch : 09 step : 0567 loss = 0.178360 accuracy= 0.937500
epoch : 09 step : 0568 loss = 0.058309 accuracy= 1.000000
epoch : 09 step : 0569 loss = 0.119584 accuracy= 1.000000
epoch : 09 step : 0570 loss = 0.162512 accuracy= 0.968750
epoch : 09 step : 0571 loss = 0.095822 accuracy= 1.000000
epoch : 09 step : 0572 loss = 0.239560 accuracy= 0.968750
epoch : 09 step : 0573 loss = 0.178601 accuracy= 0.968750
epoch : 09 step : 0574 loss = 0.137843 accuracy= 0.968750
epoch : 09 step : 0575 loss = 0.247702 accuracy= 0.937500
epoch : 09 step : 0576 loss = 0.179604 accuracy= 0.937500
epoch : 09 step : 0577 loss = 0.168943 accuracy= 0.968750
epoch : 09 step : 0578 loss = 0.239044 accuracy= 0.968750
epoch : 09 step : 0579 loss = 0.237784 accuracy= 0.937500
epoch : 09 step : 0580 loss = 0.226977 accuracy= 0.968750
epoch : 09 step : 0581 loss = 0.323871 accuracy= 0.906250
epoch : 09 ste


<Train> Loss = 0.190704 Accuracy = 0.951908
<Test> Loss = 1.746051 Accuracy = 0.553439


In [27]:
train_loss = pd.DataFrame({"train_loss":train_loss})
train_acc = pd.DataFrame({"train_acc":train_acc})
test_loss = pd.DataFrame({"test_loss":test_loss})
test_acc = pd.DataFrame({"test_acc":test_acc})
df = pd.concat([train_loss,train_acc,test_loss,test_acc], axis = 1)
df.to_csv("./Bidirectional_LSTM_4/loss_accuracy.csv", sep =",", index=False)

train_writer.close()
duration = time.time() - start_time
minute = int(duration / 60)
second = int(duration) % 60
print("%dminutes %dseconds" % (minute,second))
save_path = saver.save(sess, modelName)

print ('save_path',save_path)

112minutes 0seconds
save_path ./Bidirectional_LSTM_4/BiLSTM_model.ckpt


# 에러나는 거 지우기..

In [40]:
lis = []
for i in range(len(data)):
    if len(data[i]) == 0:
        lis.append(i)

In [48]:
df.index

Int64Index([19600,  2370, 11985, 26165, 19178, 26594, 27716, 20978, 23177,
             8799,
            ...
            13445, 11016,  5500, 16705, 16156,  7840, 20908, 12080, 28343,
            24066],
           dtype='int64', length=28798)

In [51]:
bugidx = []
for i in lis:
    bugidx.append(df.index[i])

In [52]:
df.drop(bugidx,inplace=True)

In [53]:
df.head()

,category,c_origin,jss,label
19600,재무·금융,재무·세무·IR,[맡은 일은 끝까지 완벽하게]\n고등학교 생활은 제가 책임감 있는 사람으로 성장하도...,1
2370,생산·물류,생산관리·품질관리,"스터디센터, OOOO, 초콜릿 공장 등에서 아르바이트를 경험했습니다. 각 직무마다 ...",2
11985,재무·금융,재무회계,"[단골손님]\n저는 고객이란 '신뢰를 바탕으로 한 잠재적이며, 지속적인 인간관계' ...",1
26165,영업·마케팅,마케팅·광고·분석,[내가 치킨을 사먹는다는다면?]\n\n지난 학기 때 경제이론을 실제 사례로 설명하는...,3
19178,서버·시스템,시스템프로그래머,[기본에 충실한 대인관계의 달인]\n매 학기 근로장학생을 해서 찾아간 분들에게 좋은...,6
